In [6]:
from dataclasses import dataclass
from typing import List

from autogen_core.base import MessageContext
from autogen_core.components import DefaultTopicId, RoutedAgent, default_subscription, message_handler
from autogen_core.components.code_executor import CodeExecutor, extract_markdown_code_blocks
from autogen_core.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)


@dataclass
class Message:
    content: str


@default_subscription
class Assistant(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("An assistant agent.")
        self._model_client = model_client
        self._chat_history: List[LLMMessage] = [
            SystemMessage(
                content="""Write Python script in markdown block, and it will be executed.
Always save figures to file in the current directory. Do not use plt.show()""",
            )
        ]

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        self._chat_history.append(UserMessage(content=message.content, source="user"))
        result = await self._model_client.create(self._chat_history)
        print(f"\n{'-'*80}\nAssistant:\n{result.content}")
        self._chat_history.append(AssistantMessage(content=result.content, source="assistant"))  # type: ignore
        await self.publish_message(Message(content=result.content), DefaultTopicId())  # type: ignore


@default_subscription
class Executor(RoutedAgent):
    def __init__(self, code_executor: CodeExecutor) -> None:
        super().__init__("An executor agent.")
        self._code_executor = code_executor

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        code_blocks = extract_markdown_code_blocks(message.content)
        if code_blocks:
            result = await self._code_executor.execute_code_blocks(
                code_blocks, cancellation_token=ctx.cancellation_token
            )
            print(f"\n{'-'*80}\nExecutor:\n{result.output}")
            await self.publish_message(Message(content=result.output), DefaultTopicId())

In [9]:
import tempfile

from autogen_core.application import SingleThreadedAgentRuntime
from autogen_ext.code_executors import DockerCommandLineCodeExecutor
from autogen_ext.models import OpenAIChatCompletionClient

work_dir = tempfile.mkdtemp()

# Create an local embedded runtime
runtime = SingleThreadedAgentRuntime()

async with DockerCommandLineCodeExecutor(work_dir=work_dir) as executor:
    await Assistant.register(
        runtime,
        "assistant",
        lambda: Assistant(OpenAIChatCompletionClient(model="gpt-4o")),
    )
    await Executor.register(runtime, "executor", lambda: Executor(executor))

    # Start the runtime and publish a message to the assistant.
    runtime.start()
    await runtime.publish_message(
        Message("Create a plot of NVIDA vs TSLA stock returns YTD from 2024-01-01."), DefaultTopicId()
    )
    await runtime.stop_when_idle()


--------------------------------------------------------------------------------
Assistant:
To plot Nvidia and Tesla stock returns Year-To-Date (YTD) from 2024-01-01, we'll first need to obtain historical stock data. We can use Python with libraries such as `yfinance` to fetch stock data and `matplotlib` to plot the returns.

Here's a step-by-step script to accomplish this task:

```python
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd

# Define the stock tickers
tickers = ['NVDA', 'TSLA']

# Download the stock data from Yahoo Finance from 2024-01-01 to current date
data = yf.download(tickers, start='2024-01-01', progress=False)['Adj Close']

# Calculate daily returns
returns = data.pct_change().dropna()

# Calculate cumulative returns
cumulative_returns = (1 + returns).cumprod() - 1

# Plot cumulative returns
plt.figure(figsize=(10, 6))
plt.plot(cumulative_returns.index, cumulative_returns['NVDA'], label='NVIDIA', color='green')
plt.plot(cumulative_returns.

Error processing publish message
Traceback (most recent call last):
  File "/Users/gota/Documents/projects/agent-playground/.venv/lib/python3.12/site-packages/autogen_core/application/_single_threaded_agent_runtime.py", line 385, in _process_publish
    await asyncio.gather(*responses)
  File "/Users/gota/Documents/projects/agent-playground/.venv/lib/python3.12/site-packages/autogen_core/application/_single_threaded_agent_runtime.py", line 377, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gota/Documents/projects/agent-playground/.venv/lib/python3.12/site-packages/autogen_core/components/_routed_agent.py", line 468, in on_message
    return await h(self, message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gota/Documents/projects/agent-playground/.venv/lib/python3.12/site-packages/autogen_core/components/_routed_agent.py", line 148, in wrapper
    return_value = await func(self, message, ctx)
                   ^^^^^^


--------------------------------------------------------------------------------
Assistant:
It looks like the `yfinance` module is not installed in your environment. You can install it using pip. Since I am unable to execute shell commands like `pip install` directly within this platform, I'll guide you on how you can install it:

1. Open your terminal or command prompt.
2. Run the following command: 
   ```
   pip install yfinance
   ```

Once you have installed the `yfinance` module, you should be able to run the script provided above without encountering the `ModuleNotFoundError`.

If you are running this in a Jupyter notebook, you can install the package by running the following command in a cell:

```python
!pip install yfinance
```

After installing the module, execute the script again to generate the plot.
